In [1]:
import os
import pandas as pd
from datasets import load_dataset
from pathlib import Path
import gc

BASE_DIR = r"/root"
RAW_DIR = os.path.join(BASE_DIR, "Data")
CLEANED_DIR = os.path.join(BASE_DIR, "cleaned_parquets")
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(CLEANED_DIR, exist_ok=True)


/root/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import tarfile
import os

def extract_tar_bz2_files(directory):
    for file in os.listdir(directory):
        if file.endswith(".tar.bz2"):
            filepath = os.path.join(directory, file)
            extract_dir = os.path.join(directory, file.replace(".tar.bz2", ""))
            os.makedirs(extract_dir, exist_ok=True)

            print(f"Extracting: {file}")
            with tarfile.open(filepath, "r:bz2") as tar:
                tar.extractall(path=extract_dir)


In [3]:
extract_tar_bz2_files(RAW_DIR)


Extracting: raw_meta_Clothing_Shoes_and_Jewelry.tar.bz2
Extracting: raw_review_Clothing_Shoes_and_Jewelry.tar.bz2


In [ ]:
import pandas as pd
import re
import os

def clean_and_merge_streaming(category):
    review_path = os.path.join(RAW_DIR, f"raw_review_{category}", "data.jsonl")
    meta_path = os.path.join(RAW_DIR, f"raw_meta_{category}", "data.jsonl")

    try:
        meta_df = pd.read_json(meta_path, lines=True)
    except Exception as e:
        print(f"Meta load failed for {category}: {e}")
        return

    # Preprocess metadata
    meta_df["brand"] = meta_df["details"].apply(lambda d: d.get("brand") if isinstance(d, dict) else "Unknown")
    meta_df["brand"].fillna("Unknown", inplace=True)

    save_path = os.path.join(CLEANED_DIR, f"{category}.parquet")
    chunk_iter = pd.read_json(review_path, lines=True, chunksize=100_000)
    
    for i, chunk in enumerate(chunk_iter):
        print(f"[{category}] Processing chunk {i+1}")

        # Filter and clean
        chunk = chunk.dropna(subset=["rating", "text"])
        chunk = chunk[chunk["rating"].between(1, 5)]
        chunk = chunk.drop_duplicates(subset=["user_id", "asin", "text"])
        chunk["review_length"] = chunk["text"].apply(lambda x: len(re.findall(r'\w+', str(x))))
        chunk["year"] = pd.to_datetime(chunk["timestamp"], unit='ms', errors='coerce').dt.year

        # Merge
        merged = pd.merge(chunk, meta_df, on="parent_asin", how="left")

        # Append to parquet
        if not os.path.exists(save_path):
            merged.to_parquet(save_path, index=False)
        else:
            merged.to_parquet(save_path, index=False, append=True)

    print(f"Saved {category} to {save_path}")


In [5]:
from datasets import load_from_disk
import os


def arrow_to_parquet(category):
    try:
        review_dict = load_from_disk(os.path.join(RAW_DIR, f"raw_review_{category}", f"raw_review_{category}"))
        meta_dict = load_from_disk(os.path.join(RAW_DIR, f"raw_meta_{category}", f"raw_meta_{category}"))

        # Extract the 'full' split
        review_ds = review_dict["full"]
        meta_ds = meta_dict["full"]
    except Exception as e:
        print(f"Skipping {category} — failed to load arrow or 'full' split: {e}")
        return

    # Save to Parquet
    review_path = os.path.join(CLEANED_DIR, f"{category}_review.parquet")
    meta_path = os.path.join(CLEANED_DIR, f"{category}_meta.parquet")
    print(f"Exporting {category}...")

    review_ds.to_parquet(review_path)
    meta_ds.to_parquet(meta_path)

#Detect categories with both review and meta folders
categories = [
    name.replace("raw_review_", "")
    for name in os.listdir(RAW_DIR)
    if name.startswith("raw_review_")
    and os.path.isdir(os.path.join(RAW_DIR, name))
    and os.path.isdir(os.path.join(RAW_DIR, f"raw_meta_{name.replace('raw_review_', '')}"))
]

categories = ['Clothing_Shoes_and_Jewelry']

for cat in categories:
    arrow_to_parquet(cat)


Loading dataset from disk:   0%|          | 0/38 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/31 [00:00<?, ?it/s]

Exporting Clothing_Shoes_and_Jewelry...


Creating parquet from Arrow format:   0%|          | 0/66034 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/7219 [00:00<?, ?ba/s]

In [6]:
import os


# Only look for categories with review + meta parquet files
def get_parquet_categories(cleaned_dir):
    review_files = set(
        f.replace("_review.parquet", "")
        for f in os.listdir(cleaned_dir) if f.endswith("_review.parquet")
    )
    meta_files = set(
        f.replace("_meta.parquet", "")
        for f in os.listdir(cleaned_dir) if f.endswith("_meta.parquet")
    )
    return sorted(review_files & meta_files)

categories = get_parquet_categories(CLEANED_DIR)
print("Found categories:", categories)


Found categories: ['Clothing_Shoes_and_Jewelry']


In [2]:
import duckdb
import os

def merge_parquet_to_duckdb(category, cleaned_dir):
    review_path = os.path.join(cleaned_dir, f"{category}_review.parquet")
    meta_path = os.path.join(cleaned_dir, f"{category}_meta.parquet")
    output_path = os.path.join(cleaned_dir, f"{category}_merged.parquet")

    if not os.path.exists(review_path) or not os.path.exists(meta_path):
        print(f"⚠ Skipping {category} — one of the parquet files is missing.")
        return

    print(f"Merging: {category}")

    con = duckdb.connect(database=os.path.join(cleaned_dir, "temp_duckdb.db"))
    con.execute("PRAGMA max_temp_directory_size='20GiB';")

    con.execute(f"CREATE OR REPLACE TABLE review AS SELECT * FROM '{review_path}';")

    con.execute(f"""
        CREATE OR REPLACE TABLE meta AS
        SELECT *,
            CASE
                WHEN try_cast(details AS JSON) IS NOT NULL AND json_extract(details, '$.brand') IS NOT NULL
                THEN json_extract(details, '$.brand')::STRING
                ELSE 'Unknown'
            END AS brand
        FROM '{meta_path}';
    """)

    # No deduplication
    con.execute(f"""
        CREATE OR REPLACE TABLE cleaned AS
        SELECT
            r.user_id,
            r.asin,
            r.parent_asin,
            r.rating,
            r.text,
            r.verified_purchase,
            r.helpful_vote,
            array_length(string_split(r.text, ' ')) AS review_length,
             strftime(
                    CASE 
                        WHEN typeof(r.timestamp) = 'VARCHAR' THEN CAST(r.timestamp AS TIMESTAMP)
                        ELSE to_timestamp(CAST(r.timestamp AS DOUBLE) / 1000.0)
                    END,
                    '%Y'
                )::INTEGER AS year,
            m.brand,
            m.main_category,
            m.title,
            m.average_rating,
            m.rating_number,
            m.price,
            '{category}' AS category
        FROM review r
        LEFT JOIN meta m ON r.parent_asin = m.parent_asin
        WHERE r.rating BETWEEN 1 AND 5 AND r.text IS NOT NULL;
    """)

    con.execute(f"COPY cleaned TO '{output_path}' (FORMAT PARQUET);")
    con.close()
    print(f"Saved merged and cleaned data → {output_path}")

In [9]:
import os
import glob
import duckdb

def deduplicate_large_parquet_chunked(con, input_path, dedup_columns, order_column='year', chunk_size=250_000):
    print(f"🧠 Using chunked deduplication for large file: {os.path.basename(input_path)}")

    con.execute("DROP TABLE IF EXISTS temp_all;")
    con.execute(f"""
        CREATE TABLE temp_all AS
        SELECT * FROM read_parquet('{input_path}', union_by_name=true)
        LIMIT 0;
    """)

    offset = 0
    while True:
        chunk_df = con.execute(f"""
            SELECT * FROM read_parquet('{input_path}', union_by_name=true)
            LIMIT {chunk_size} OFFSET {offset}
        """).fetchdf()

        if chunk_df.empty:
            break

        con.register("chunk", chunk_df)
        con.execute("INSERT INTO temp_all SELECT * FROM chunk")
        print(f"✅ Chunk {offset // chunk_size + 1} inserted ({len(chunk_df)} rows)")
        offset += chunk_size

    dedup_key = ', '.join(dedup_columns)
    con.execute("DROP TABLE IF EXISTS deduplicated;")
    con.execute(f"""
        CREATE TABLE deduplicated AS
        SELECT * EXCLUDE(row_num)
        FROM (
            SELECT *,
                ROW_NUMBER() OVER (
                    PARTITION BY {dedup_key}
                    ORDER BY {order_column}
                ) AS row_num
            FROM temp_all
        )
        WHERE row_num = 1;
    """)

In [6]:
def deduplicate_folder_merged_files(input_dir, output_dir, dedup_columns, order_column='year', chunk_threshold_gb=2.0):
    os.makedirs(output_dir, exist_ok=True)
    all_parquets = glob.glob(os.path.join(input_dir, "*_merged.parquet"))

    print(f"📁 Found {len(all_parquets)} '_merged' files to deduplicate...\n")

    for input_path in all_parquets:
        filename = os.path.basename(input_path)
        name_wo_ext = filename.replace("_merged.parquet", "")
        output_path = os.path.join(output_dir, f"{name_wo_ext}_deduped.parquet")

        if os.path.exists(output_path):
            print(f"⏩ Skipping (already deduplicated): {filename}")
            continue

        file_size_gb = os.path.getsize(input_path) / (1024 ** 3)
        con = duckdb.connect()

        try:
            if file_size_gb >= chunk_threshold_gb:
                deduplicate_large_parquet_chunked(
                    con,
                    input_path,
                    dedup_columns=dedup_columns,
                    order_column=order_column,
                    chunk_size=250_000
                )
            else:
                print(f"🧼 Deduplicating: {filename} (size: {file_size_gb:.2f} GB)")
                dedup_key = ', '.join(dedup_columns)

                con.execute("DROP TABLE IF EXISTS deduplicated;")
                con.execute(f"""
                    CREATE TABLE deduplicated AS
                    SELECT * EXCLUDE(row_num)
                    FROM (
                        SELECT *,
                            ROW_NUMBER() OVER (
                                PARTITION BY {dedup_key}
                                ORDER BY {order_column}
                            ) AS row_num
                        FROM read_parquet('{input_path}', union_by_name=true)
                    )
                    WHERE row_num = 1;
                """)

            con.execute(f"COPY deduplicated TO '{output_path}' (FORMAT PARQUET);")
            print(f"✅ Deduplicated and saved to: {output_path}\n")

        except Exception as e:
            print(f"❌ Error with {filename}: {e}\n")
        finally:
            con.close()

    print("🏁 All deduplication complete.")

In [3]:
categories = ['Clothing_Shoes_and_Jewelry']
# 'Beauty_and_Personal_Care', 'Electronics', 'Books', 'Clothing_Shoes_and_Jewelry', 'All_Beauty', 'Amazon_Fashion',  'Appliances', 'Arts_Crafts_and_Sewing',  'Baby_Products', 'CDs_and_Vinyl',  'Digital_Music', 'Gift_Cards' 'Cell_Phones_and_Accessories', 

# deduplicate_folder_merged_files(
#     CLEANED_DIR,
#     CLEANED_DIR,
#     dedup_columns=["user_id", "asin", "text"],
#     order_column="year",
#     chunk_threshold_gb=2.0  # Files over 2 GB will trigger chunking
# )

for cat in categories:
    merge_parquet_to_duckdb(cat, CLEANED_DIR)
    # deduplicate_all_parquets(CLEANED_DIR)
    gc.collect()


Merging: Clothing_Shoes_and_Jewelry


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Saved merged and cleaned data → /root/cleaned_parquets/Clothing_Shoes_and_Jewelry_merged.parquet


In [ ]:
import duckdb
import os

def merge_parquet_to_duckdb(category, cleaned_dir):
    review_path = os.path.join(cleaned_dir, f"{category}_review.parquet")
    meta_path = os.path.join(cleaned_dir, f"{category}_meta.parquet")
    output_path = os.path.join(cleaned_dir, f"{category}_merged.parquet")

    if not os.path.exists(review_path) or not os.path.exists(meta_path):
        print(f"⚠️ Skipping {category} — one of the parquet files is missing.")
        return

    print(f"🔄 Merging: {category}")

    con = duckdb.connect(database=os.path.join(cleaned_dir, "temp_duckdb.db"))
    con.execute("PRAGMA max_temp_directory_size='64GiB';")

    con.execute(f"""
        CREATE OR REPLACE TABLE review AS SELECT * FROM '{review_path}';
    """)

    con.execute(f"""
        CREATE OR REPLACE TABLE meta AS
        SELECT *,
            CASE
                WHEN try_cast(details AS JSON) IS NOT NULL AND json_extract(details, '$.brand') IS NOT NULL
                THEN json_extract(details, '$.brand')::STRING
                ELSE 'Unknown'
            END AS brand
        FROM '{meta_path}';
    """)

    con.execute(f"""
        CREATE OR REPLACE TABLE cleaned AS
        SELECT *
        FROM (
            SELECT
                r.user_id,
                r.asin,
                r.parent_asin,
                r.rating,
                r.text,
                r.verified_purchase,
                r.helpful_vote,
                array_length(string_split(r.text, ' ')) AS review_length,
                strftime(
                    CASE 
                        WHEN typeof(r.timestamp) = 'VARCHAR' THEN CAST(r.timestamp AS TIMESTAMP)
                        ELSE to_timestamp(CAST(r.timestamp AS DOUBLE) / 1000.0)
                    END,
                    '%Y'
                )::INTEGER AS year,
                m.brand,
                m.main_category,
                m.title,
                m.average_rating,
                m.rating_number,
                m.price,
                '{category}' AS category,
                ROW_NUMBER() OVER (
                    PARTITION BY r.user_id, r.asin, r.text
                    ORDER BY r.timestamp
                ) AS row_num
            FROM review r
            LEFT JOIN meta m ON r.parent_asin = m.parent_asin
            WHERE r.rating BETWEEN 1 AND 5 AND r.text IS NOT NULL
        )
        WHERE row_num = 1;
    """)

    con.execute(f"""
        COPY cleaned TO '{output_path}' (FORMAT PARQUET);
    """)

    con.close()
    print(f"Saved merged and cleaned data → {output_path}")


In [ ]:
import duckdb
import os
import hashlib

def deduplicate_reviews(category, cleaned_dir):
    intermediate_path = os.path.join(cleaned_dir, f"{category}_merged_raw.parquet")
    output_path = os.path.join(cleaned_dir, f"{category}_merged.parquet")
    temp_db_path = os.path.join(cleaned_dir, "temp_duckdb.db")

    if not os.path.exists(intermediate_path):
        print(f"⚠️ Skipping {category} — intermediate file missing.")
        return

    print(f"🧹 Deduplicating in chunks: {category}")

    con = duckdb.connect(database=temp_db_path)

    con.execute("DROP TABLE IF EXISTS temp_reviews;")
    con.execute(f"CREATE TABLE temp_reviews AS SELECT * FROM read_parquet('{intermediate_path}');")

    hex_chars = '0123456789abcdef'
    temp_outputs = []

    for char in hex_chars:
        print(f"🧩 Processing chunk for user_id hash prefix '{char}'")
        temp_output_chunk = os.path.join(cleaned_dir, f"tmp_{category}_{char}.parquet")
        temp_outputs.append(temp_output_chunk)

        con.execute(f"""
            COPY (
                SELECT *
                FROM (
                    SELECT *,
                        ROW_NUMBER() OVER (
                            PARTITION BY user_id, asin, text
                            ORDER BY timestamp
                        ) AS row_num
                    FROM temp_reviews
                    WHERE substr(md5(user_id), 1, 1) = '{char}'
                )
                WHERE row_num = 1
            ) TO '{temp_output_chunk}' (FORMAT PARQUET);
        """)

    # Merge all chunk outputs
    print(f"🧬 Merging deduplicated chunks for: {category}")
    union_sql = " UNION ALL ".join([
    f"(SELECT * FROM read_parquet('{p}'))" for p in temp_outputs])
    con.execute(f"COPY ({union_sql}) TO '{output_path}' (FORMAT PARQUET);")

    # Cleanup
    for temp_file in temp_outputs:
        os.remove(temp_file)

    con.close()
    print(f"✅ Saved deduplicated data → {output_path}")


In [3]:
import os
import glob
import duckdb

def deduplicate_all_parquets(folder_path):
    parquet_files = glob.glob(os.path.join(folder_path, "*_merged.parquet"))
    
    if not parquet_files:
        print("No merged parquet files found.")
        return

    con = duckdb.connect()

    for file_path in parquet_files:
        filename = os.path.basename(file_path)
        output_path = os.path.join(folder_path, filename.replace("_merged.parquet", "_deduped.parquet"))
        
        print(f"Deduplicating: {filename}")

        con.execute(f"""
            CREATE OR REPLACE TABLE deduplicated AS
            SELECT * EXCLUDE(row_num)
            FROM (
                SELECT *,
                    ROW_NUMBER() OVER (
                        PARTITION BY user_id, asin, text
                        ORDER BY year
                    ) AS row_num
                FROM read_parquet('{file_path}')
            )
            WHERE row_num = 1;
        """)

        con.execute(f"COPY deduplicated TO '{output_path}' (FORMAT PARQUET);")
        print(f"Saved deduplicated file → {output_path}")

    con.close()